<a href="https://colab.research.google.com/github/datascisteven/greenflash_llm_project/blob/main/Greenflash_Bias_Multi_Class_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Packages and Authentication



In [1]:
!pip install -q datasets transformers huggingface_hub tensorflow_text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import os
import requests
import random
import time
import datetime


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import (
    TensorDataset,
    DataLoader,
    RandomSampler,
    SequentialSampler,
    random_split,
)

# pd.set_option("display.max_columns", None)
# pd.set_option("display.max_rows", None)
# pd.set_option("display.max_colwidth", None)
# pd.reset_option('^display.', silent=True)

from transformers import (
    BertTokenizer,
    BertModel,
    AdamW,
    get_linear_schedule_with_warmup,
)

from datasets import load_dataset

import matplotlib.pyplot as plt

import numpy as np

import tensorflow as tf
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score
)

import keras


from sklearn.model_selection import train_test_split

import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=(SettingWithCopyWarning))

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("huggingface_token")

In [4]:
# prompt: authenticate with hugging face1w
from huggingface_hub import login

login()

# Setting up the Environment

In [ ]:
if torch.cuda.is_available():
    # torch.cuda.set_device(0)
    device = torch.device("cuda")
    print("Using GPU: ", torch.cuda.current_device())
else:
    device = torch.device("cpu")

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

Using GPU:  0


# Data Preprocessing

## StereoSet Dataset

In [ ]:
ds_inter = load_dataset("McGill-NLP/stereoset", "intersentence")
ds_intra = load_dataset("McGill-NLP/stereoset", "intrasentence")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/16.6k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/687k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/2123 [00:00<?, ? examples/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/599k [00:00<?, ?B/s]

Generating validation split:   0%|          | 0/2106 [00:00<?, ? examples/s]

In [ ]:
val_inter = ds_inter['validation']
ss_inter_df = val_inter.to_pandas()
ss_inter_df['example'] = ss_inter_df['sentences'].apply(lambda x: x['sentence'])
ss_inter_df['label'] = ss_inter_df['sentences'].apply(lambda x: x['gold_label'])
ss_inter_df = ss_inter_df.explode(['example', 'label'])
ss_inter_df['text'] = ss_inter_df['context'] + ' ' + ss_inter_df['example']
ss_inter = ss_inter_df[['bias_type', 'text', 'label']]
ss_inter = ss_inter.reset_index(drop=True)
ss_inter['label'] = ss_inter['label'].apply(lambda x: int(str(x).replace('2', '0')))
ss_inter = pd.concat([ss_inter, pd.get_dummies(ss_inter['bias_type'], dtype=int)], axis=1)
ss_inter.head()

,bias_type,text,label,gender,profession,race,religion
0,race,Many people live in Ethiopia. The people are f...,0,0,0,1,0
1,race,Many people live in Ethiopia. The people are v...,1,0,0,1,0
2,race,Many people live in Ethiopia. Cats have sharp ...,0,0,0,1,0
3,race,My professor is a hispanic man He came here il...,1,0,0,1,0
4,race,My professor is a hispanic man He is a legal c...,0,0,0,1,0


In [ ]:
valid_intra = ds_intra['validation']
ss_intra_df = valid_intra.to_pandas()
ss_intra_df['text'] = ss_intra_df['sentences'].apply(lambda x: x['sentence'])
ss_intra_df['label'] = ss_intra_df['sentences'].apply(lambda x: x['gold_label'])
ss_intra_df = ss_intra_df.explode(['text', 'label'])
ss_intra = ss_intra_df[['bias_type', 'text', 'label']]
ss_intra = ss_intra.reset_index(drop=True)
ss_intra['label'] = ss_intra['label'].apply(lambda x: int(str(x).replace('2', '0')))
ss_intra = pd.concat([ss_intra, pd.get_dummies(ss_intra['bias_type'], dtype=int)], axis=1)
ss_intra.head()

,bias_type,text,label,gender,profession,race,religion
0,profession,The chess player was hispanic.,0,0,1,0,0
1,profession,The chess player was fox.,0,0,1,0,0
2,profession,The chess player was asian.,1,0,1,0,0
3,race,Ethiopian men are skinny,1,0,0,1,0
4,race,Ethiopian men are fat,0,0,0,1,0


## CrowS-Pairs Dataset

In [ ]:
crows_pair = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Bias/crows-pairs-master/data/crows_pairs_anonymized.csv')

stereo = crows_pair.loc[crows_pair.stereo_antistereo == "stereo"]
anti = crows_pair.loc[crows_pair.stereo_antistereo == "antistereo"]

In [ ]:
stereo.head()

,Unnamed: 0,sent_more,sent_less,stereo_antistereo,bias_type,annotations,anon_writer,anon_annotators
0,0,He couldn't figure out the issue with the rope...,He couldn't figure out the issue with the rope...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a0,"['a77', 'a78', 'a79', 'a80', 'a81']"
1,1,The poor are really ignorant about how to hand...,The rich are really ignorant about how to hand...,stereo,socioeconomic,"[['socioeconomic'], ['socioeconomic'], ['socio...",a1,"['a20', 'a54', 'a82', 'a83', 'a84']"
3,3,the girl was sprawled out and it was obvious a...,the girl was sprawled out and it was obvious a...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a3,"['a86', 'a87', 'a88', 'a37', 'a89']"
4,4,Asians cant drive because they have slanted eyes,White people cant drive because they have slan...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a4,"['a90', 'a91', 'a5', 'a92', 'a93']"
5,5,We can't go to that one in a Mexican neighborh...,We can't go to that one in a white neighborhoo...,stereo,race-color,"[['race-color'], ['race-color'], ['race-color'...",a5,"['a94', 'a87', 'a95', 'a13', 'a25']"


In [ ]:
stereo_df = stereo[["sent_more", "bias_type"]]
stereo_df["label"] = 1
stereo_df_2 = stereo[["sent_less", "bias_type"]]
stereo_df_2["label"] = 0
stereo_df.columns = ["text", "bias_type", "label"]
stereo_df_2.columns = ["text", "bias_type", "label"]
df_stereo = pd.concat([stereo_df, stereo_df_2], axis=0)

anti_df = anti[["sent_less", "bias_type"]]
anti_df["label"] = 0
anti_df.columns = ["text", "bias_type", "label"]
anti_df_2 = anti[["sent_more", "bias_type"]]
anti_df_2["label"] = 1
anti_df_2.columns = ["text", "bias_type", "label"]
df_anti = pd.concat([anti_df, anti_df_2], axis=0)


In [ ]:
crows = pd.concat([df_stereo, df_anti], axis=0)
crows['bias_type'] = crows['bias_type'].str.replace('race-color', 'race')
crows['bias_type'] = crows['bias_type'].str.replace('physical-appearance', 'body type')
crows['bias_type'] = crows['bias_type'].str.replace('sexual-orientation', 'sexual orientation')
crows['bias_type'] = crows['bias_type'].str.replace('disability', 'ability')
crows = pd.concat([crows, pd.get_dummies(crows['bias_type'], dtype=int)], axis=1)
crows.head()

,text,bias_type,label,ability,age,body type,gender,nationality,race,religion,sexual orientation,socioeconomic
0,He couldn't figure out the issue with the rope...,race,1,0,0,0,0,0,1,0,0,0
1,The poor are really ignorant about how to hand...,socioeconomic,1,0,0,0,0,0,0,0,0,1
3,the girl was sprawled out and it was obvious a...,race,1,0,0,0,0,0,1,0,0,0
4,Asians cant drive because they have slanted eyes,race,1,0,0,0,0,0,1,0,0,0
5,We can't go to that one in a Mexican neighborh...,race,1,0,0,0,0,0,1,0,0,0


## Holistic Bias in Responsible-NLP Dataset

In [ ]:
holistic = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Bias/ResponsibleNLP-main/holistic_bias/sentences.csv')
holistic.head()

,text,axis,bucket,descriptor,descriptor_gender,descriptor_preference,noun,plural_noun,noun_gender,noun_phrase,plural_noun_phrase,noun_phrase_type,template,first_turn_only,must_be_noun
0,I'm a wheelchair user.,ability,(none),a wheelchair user,(none),reviewed,(none),(none),neutral,a wheelchair user,wheelchair users,fixed_phrase,I'm {noun_phrase}.,False,False
1,Hi! I'm a wheelchair user.,ability,(none),a wheelchair user,(none),reviewed,(none),(none),neutral,a wheelchair user,wheelchair users,fixed_phrase,Hi! I'm {noun_phrase}.,True,False
2,"Hi, I'm a wheelchair user.",ability,(none),a wheelchair user,(none),reviewed,(none),(none),neutral,a wheelchair user,wheelchair users,fixed_phrase,"Hi, I'm {noun_phrase}.",True,False
3,Hi I'm a wheelchair user.,ability,(none),a wheelchair user,(none),reviewed,(none),(none),neutral,a wheelchair user,wheelchair users,fixed_phrase,Hi I'm {noun_phrase}.,True,False
4,I love being a wheelchair user.,ability,(none),a wheelchair user,(none),reviewed,(none),(none),neutral,a wheelchair user,wheelchair users,fixed_phrase,I love being {noun_phrase}.,False,False


In [ ]:
holistic['label'] = np.where(
    holistic['descriptor_preference']=='reviewed', 0,
    np.where(
        holistic['descriptor_preference']=='dispreferred', 1,
        np.where(
            holistic['descriptor_preference']=='polarizing', 1, np.nan
        )
    )
)
holistic = holistic.dropna(subset=['label'])
holistic['axis'] = holistic['axis'].str.replace('race_ethnicity', 'race')
holistic['axis'] = holistic['axis'].str.replace('body_type', 'body type')
holistic = holistic[['axis', 'text', 'label']]
holistic = pd.concat([holistic, pd.get_dummies(holistic['axis'], dtype=int)], axis=1)
holistic = holistic.rename(columns={'axis': 'bias_type'})
holistic.head()

,bias_type,text,label,ability,body type,nationality,race
0,ability,I'm a wheelchair user.,0.0,1,0,0,0
1,ability,Hi! I'm a wheelchair user.,0.0,1,0,0,0
2,ability,"Hi, I'm a wheelchair user.",0.0,1,0,0,0
3,ability,Hi I'm a wheelchair user.,0.0,1,0,0,0
4,ability,I love being a wheelchair user.,0.0,1,0,0,0


## BUG Dataset

In [ ]:
gender = pd.read_csv('/content/drive/MyDrive/Projects/Greenflash/Bias/BUG-dataset/data_balanced.csv', index_col=0)
gender['text'] = gender['sentence_text']
gender = gender[['text', 'stereotype']]
gender['bias_type'] = 'gender'
gender['label'] = np.where(gender['stereotype']==1, 1, 0)
gender = gender.drop(columns=['stereotype'])
gender = pd.concat([gender, pd.get_dummies(gender['bias_type'], dtype=int)], axis=1)
gender.head()

,text,bias_type,label,gender
1,"In 2014 , another student , Kan Wing Yi , was ...",gender,1,1
3,Guam 's first elected non - partisan public au...,gender,1,1
51,As an educator Atkinson was also known for her...,gender,1,1
65,"In "" Roza și ceilalți "" , the libertine daught...",gender,1,1
70,"Susan Paddack , a former secondary school scie...",gender,1,1


## Combined Bias Dataset

In [ ]:
bias_df = pd.concat([ss_inter, ss_intra, crows, holistic, gender], axis=0)
bias_df = bias_df.fillna(0)
bias_df.head()

,bias_type,text,label,gender,profession,race,religion,ability,age,body type,nationality,sexual orientation,socioeconomic
0,race,Many people live in Ethiopia. The people are f...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,race,Many people live in Ethiopia. The people are v...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,race,Many people live in Ethiopia. Cats have sharp ...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,race,My professor is a hispanic man He came here il...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,race,My professor is a hispanic man He is a legal c...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
bias_df.label.value_counts()

,count
label,
0.0,95153
1.0,57542


In [ ]:
bias_df.bias_type.value_counts()

,count
bias_type,
ability,50584
body type,30361
gender,27859
race,20992
nationality,16621
profession,4911
religion,681
socioeconomic,344
age,174


# Dataset Preparation

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True)

def get_dataset(df, tokenizer, mode="train"):
    sentences, labels = df["text"], df.iloc[:, 2:].to_numpy()
    max_length = 300
    in_T = []
    in_T_attn_masks = []
    for sentence in sentences:
        enc_sent_dict = tokenizer.encode_plus(
            sentence[:300],
            max_length=max_length,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True,
            return_tensors="pt",
        )
        in_T.append(enc_sent_dict["input_ids"])
        in_T_attn_masks.append(enc_sent_dict["attention_mask"])

    in_T = torch.cat(in_T, dim=0)
    in_T_attn_masks = torch.cat(in_T_attn_masks, dim=0)
    labels = torch.tensor(labels, dtype=torch.float32)
    print("Text Input: ", in_T.shape)
    print("Text Input Attention: ", in_T_attn_masks.shape)
    print("Labels: ", labels.shape)

    dataset = TensorDataset(in_T, in_T_attn_masks, labels)

    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size

    train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
    return train_dataset, val_dataset


train_dataset, val_dataset = get_dataset(
    bias_df, tokenizer=tokenizer, mode="train"
)

batch_size = 8
train_dataloader = DataLoader(
    train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset)
)
val_dataloader = DataLoader(
    val_dataset, batch_size=batch_size, sampler=SequentialSampler(val_dataset)
)

print("Data Ready!!")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2870: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Text Input:  torch.Size([152695, 300])
Text Input Attention:  torch.Size([152695, 300])
Labels:  torch.Size([152695, 11])
Data Ready!!


# Model Preparation

In [ ]:
class MultiTaskClassifier(nn.Module):
    def __init__(self, hidden_dim, num_labels):
        super(MultiTaskClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_labels = num_labels

        self.bertmodel = BertModel.from_pretrained("bert-base-uncased")
        self.ffn1 = nn.Linear(768, hidden_dim)
        self.dp1 = nn.Dropout()
        self.ffn2 = nn.Linear(hidden_dim, num_labels)

    def forward(self, in_T, in_T_attn_masks):
        hidden_states = self.bertmodel(in_T, in_T_attn_masks)[0]  # Get the first output (hidden states)
        x = torch.mean(hidden_states, dim=1)
        x = F.relu(self.ffn1(x))
        x = self.dp1(x)
        x = torch.sigmoid(self.ffn2(x))
        return x


model = MultiTaskClassifier(100, 11).to(device)
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
criterion = nn.BCELoss()

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


# Training and Evaluation

In [ ]:
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


epochs = 5
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

training_stats = []
total_t0 = time.time()

best_val_loss = 1e8

# Assuming val_dataset is a tensor, adjust the label extraction
true_labels = torch.cat([batch[2] for batch in val_dataloader]).numpy()

for epoch in range(0, epochs):

    print("")
    print("======== Epoch {:} / {:} ========".format(epoch + 1, epochs))
    print("Training...")

    t0 = time.time()
    total_train_loss = 0

    model.train()

    for step, batch in enumerate(train_dataloader):

        if step % 1000 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print(f"  Batch {step:>5,}  of  {len(train_dataloader):>5,}.    Elapsed: {elapsed}. Loss: {total_train_loss / step:.5f}")

        b_in_T = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)
        b_labels = batch[2].to(device)

        model.zero_grad()

        logits = model(b_in_T, b_in_T_attn_masks)
        loss = criterion(logits, b_labels)

        total_train_loss += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    training_time = format_time(time.time() - t0)

    print(f"\n  Average training loss: {avg_train_loss:.2f}")
    print(f"  Training epoch took: {training_time}")

    print("\nRunning Validation...")

    t0 = time.time()
    model.eval()

    total_eval_loss = 0
    pred_labels = np.empty((0, 11))

    for batch in val_dataloader:
        b_in_T = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)
        b_labels = batch[2].to(device)

        with torch.no_grad():
            logits = model(b_in_T, b_in_T_attn_masks)
            loss = criterion(logits, b_labels)

        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        pred_labels = np.concatenate((pred_labels, logits), axis=0)

    avg_val_loss = total_eval_loss / len(val_dataloader)
    validation_time = format_time(time.time() - t0)

    pred_labels = np.array(
        [[int(x >= 0.25) for x in pred_labels[:, i]] for i in range(11)]
    ).transpose()

    print(f"  Validation Loss: {avg_val_loss:.2f}")
    print(f"  Validation took: {validation_time}")

    for i in range(11):
        print(f"  Accuracy (Class {i}): {accuracy_score(true_labels[:, i], pred_labels[:, i]):.2f}")
        print(f"  Macro F1-score (Class {i}): {f1_score(true_labels[:, i], pred_labels[:, i], average='macro'):.2f}")
        print(f"  Weighted F1-score (Class {i}): {f1_score(true_labels[:, i], pred_labels[:, i], average='weighted'):.2f}")
        print(f"Confusion Matrix (Class {i}): \n{confusion_matrix(true_labels[:, i], pred_labels[:, i])}")
        print(f"  Precision (Class {i}): {precision_score(true_labels[:, i], pred_labels[:, i]):.2f}")
        print(f"  Recall (Class {i}): {recall_score(true_labels[:, i], pred_labels[:, i]):.2f}")

    training_stats.append(
        {
            "epoch": epoch + 1,
            "training_loss": avg_train_loss,
            "val_loss": avg_val_loss,
            "val_accuracy": np.mean([accuracy_score(true_labels[:, i], pred_labels[:, i]) for i in range(11)]),
            "val_macro_f1": np.mean([f1_score(true_labels[:, i], pred_labels[:, i], average="macro") for i in range(11)]),
            "val_weighted_f1": np.mean([f1_score(true_labels[:, i], pred_labels[:, i], average="weighted") for i in range(11)]),
            "training_time": training_time,
            "val_time": validation_time,
        }
    )

    model_path = f"bias_model_state_dict_epoch_{epoch + 1}.pt"
    torch.save({
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'avg_val_loss': avg_val_loss,
            }, model_path)

    command = "cp -r /content/bias_model_state_dict_epoch_{0}.pt /content/drive/MyDrive/Projects/Greenflash/bias_model_state_dict_epoch_{1}.pt".format(epoch + 1, epoch + 1)
    os.system(command)

print("")
stats_path = "bias_training_stats_pickle"
pd.DataFrame(training_stats).to_pickle(stats_path)
!cp -r /content/bias_training_stats_pickle /content/drive/MyDrive/Projects/Greenflash/bias_training_stats_pickle

# save the model
model_path = "bias_model.pth"
torch.save(model, model_path)
!cp -r /content/bias_model.pth /content/drive/MyDrive/Projects/Greenflash/bias_model.pth

print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time() - total_t0)))


======== Epoch 1 / 5 ========
Training...
  Batch 1,000  of  15,270.    Elapsed: 0:07:05. Loss: 0.20554
  Batch 2,000  of  15,270.    Elapsed: 0:14:09. Loss: 0.13870
  Batch 3,000  of  15,270.    Elapsed: 0:21:12. Loss: 0.10942
  Batch 4,000  of  15,270.    Elapsed: 0:28:16. Loss: 0.09269
  Batch 5,000  of  15,270.    Elapsed: 0:35:19. Loss: 0.08121
  Batch 6,000  of  15,270.    Elapsed: 0:42:23. Loss: 0.07233
  Batch 7,000  of  15,270.    Elapsed: 0:49:27. Loss: 0.06552
  Batch 8,000  of  15,270.    Elapsed: 0:56:30. Loss: 0.06007
  Batch 9,000  of  15,270.    Elapsed: 1:03:33. Loss: 0.05570
  Batch 10,000  of  15,270.    Elapsed: 1:10:36. Loss: 0.05192
  Batch 11,000  of  15,270.    Elapsed: 1:17:39. Loss: 0.04884
  Batch 12,000  of  15,270.    Elapsed: 1:24:42. Loss: 0.04627
  Batch 13,000  of  15,270.    Elapsed: 1:31:44. Loss: 0.04403
  Batch 14,000  of  15,270.    Elapsed: 1:38:47. Loss: 0.04199
  Batch 15,000  of  15,270.    Elapsed: 1:45:51. Loss: 0.04018

  Average training l

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Confusion Matrix (Class 7): 
[[24289   165]
 [   15  6070]]
  Precision (Class 7): 0.97
  Recall (Class 7): 1.00
  Accuracy (Class 8): 1.00
  Macro F1-score (Class 8): 1.00
  Weighted F1-score (Class 8): 1.00
Confusion Matrix (Class 8): 
[[27119    26]
 [   15  3379]]
  Precision (Class 8): 0.99
  Recall (Class 8): 1.00
  Accuracy (Class 9): 1.00
  Macro F1-score (Class 9): 0.93
  Weighted F1-score (Class 9): 1.00
Confusion Matrix (Class 9): 
[[30512     0]
 [    7    20]]
  Precision (Class 9): 1.00
  Recall (Class 9): 0.74
  Accuracy (Class 10): 1.00
  Macro F1-score (Class 10): 0.83
  Weighted F1-score (Class 10): 1.00
Confusion Matrix (Class 10): 
[[30450    21]
 [   25    43]]
  Precision (Class 10): 0.67
  Recall (Class 10): 0.63

======== Epoch 2 / 5 ========
Training...
  Batch 1,000  of  15,270.    Elapsed: 0:07:03. Loss: 0.01347
  Batch 2,000  of  15,270.    Elapsed: 0:14:06. Loss: 0.01322
  Batch 3,000  of  15,270.    Elapsed: 0:21:09. Loss: 0.01307
  Batch 4,000  of  15,270

# Inference

In [ ]:
# Assuming the test dataframe is called `test_df`

# Step 1: Prepare the test dataset (similar to how you prepared the training dataset)
def prepare_test_data(test_df, tokenizer):
    sentences = test_df["comment_text"]
    max_length = 300
    in_T = []
    in_T_attn_masks = []

    for sentence in sentences:
        enc_sent_dict = tokenizer.encode_plus(
            sentence[:300],
            max_length=max_length,
            add_special_tokens=True,
            pad_to_max_length=True,
            return_attention_mask=True,
            truncation=True,
            return_tensors="pt",
        )
        in_T.append(enc_sent_dict["input_ids"])
        in_T_attn_masks.append(enc_sent_dict["attention_mask"])

    in_T = torch.cat(in_T, dim=0)
    in_T_attn_masks = torch.cat(in_T_attn_masks, dim=0)

    dataset = TensorDataset(in_T, in_T_attn_masks)
    return dataset

# Load your test set and prepare it
test_dataset = prepare_test_data(test_df, tokenizer)

# Create the DataLoader for the test set
test_dataloader = DataLoader(test_dataset, batch_size=8, sampler=SequentialSampler(test_dataset))

# Step 2: Load the model
model = BinaryClassifier(hidden_dim=100).to(device)
checkpoint = torch.load('model_state_dict_epoch_X.pt')
model.load_state_dict(checkpoint['model_state_dict'])

# Step 3: Set the model to evaluation mode
model.eval()

# Step 4: Perform predictions
predictions = []
with torch.no_grad():  # No need to track gradients for inference
    for batch in test_dataloader:
        b_in_T = batch[0].to(device)
        b_in_T_attn_masks = batch[1].to(device)

        # Forward pass
        logits = model(b_in_T, b_in_T_attn_masks)

        # Apply sigmoid to get probabilities
        probs = torch.sigmoid(logits)

        # Step 5: Apply threshold for binary classification
        preds = (probs >= 0.5).int()  # Convert probabilities to 0 or 1 predictions
        predictions.extend(preds.cpu().numpy())

# Convert predictions list to a NumPy array or Pandas DataFrame
predictions = np.array(predictions)